In [291]:
%load_ext autoreload
%autoreload 2

from report_analysis_functions import *

import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [370]:
df = get_model_results()

Linear regression Extrapolation with 475 rows does not have all metrics.
Expected 13 unique values for 'dds_metric' column, got 5
Missing:
	- total_received_samples
	- total_lost_samples_percentage
	- avg_lost_samples_percentage
	- total_throughput_mbps
	- avg_lost_samples
	- avg_samples_per_sec
	- total_samples_per_sec
	- avg_received_samples_percentage

Linear regression Interpolation with 760 rows does not have all metrics.
Expected 13 unique values for 'dds_metric' column, got 8
Missing:
	- avg_received_samples
	- avg_throughput_mbps
	- latency_us
	- total_lost_samples
	- total_received_samples_percentage

Random forests Extrapolation with 665 rows does not have all metrics.
Expected 13 unique values for 'dds_metric' column, got 7
Missing:
	- avg_throughput_mbps
	- avg_received_samples_percentage
	- total_throughput_mbps
	- avg_lost_samples
	- avg_samples_per_sec
	- total_received_samples_percentage

Random forests Interpolation with 570 rows does not have all metrics.
Expected 13 

In [327]:
data = df.copy()

data = data[
    data['error_metric'].isin(
        [
            'r2',
            'rmse'
        ]
    )
]

data = data.sort_values(
    by=[
        'model_type', 
        'int_or_ext',
        'dds_metric', 
        'output_variable',
        'error_metric'
    ],
    ascending=[
        True,
        False,
        True,
        True,
        True
    ]
)

grouped = data.groupby(
    [
        'model_type', 
        'int_or_ext', 
        'dds_metric', 
        'filename'
    ]
)

latex_file_output = ""

new_df = pd.DataFrame()

for (model_type, int_or_ext, dds_metric, filename), group in grouped:
    
    for error_metric in group['error_metric'].unique():
        error_metric_group = group[group['error_metric'] == error_metric]

        new_filename = "_".join(filename.replace(".json", "").split("_")[2:])
        title = f"{model_type} {int_or_ext.capitalize()} {dds_metric} {new_filename} {error_metric.upper()}"
        
        error_metric_group = error_metric_group[
            [
                'output_variable', 
                'train_value', 
                'test_value'
            ]
        ] 
        error_metric_group['output_variable'] = error_metric_group['output_variable'].apply(
            lambda x: x.replace(f"{dds_metric}_", "")
        )

        non_percentile_stats = ['mean', 'std', 'min', 'max']

        non_percentile_df = error_metric_group[
            error_metric_group['output_variable'].isin(non_percentile_stats)
        ]
        non_percentile_df['output_variable'] = pd.Categorical(
            non_percentile_df['output_variable'], 
            categories=non_percentile_stats
        )
        non_percentile_df.sort_values(
            by='output_variable', 
            inplace=True, 
            ignore_index=True
        )
        non_percentile_df['output_variable'] = non_percentile_df['output_variable'].apply(
            lambda x: format_non_percentile_stat(x)
        )

        percentile_df = error_metric_group[
            ~error_metric_group['output_variable'].isin(
                non_percentile_stats
            )
        ]
        percentile_df['output_variable'] = percentile_df['output_variable'].astype(int)
        percentile_df.sort_values(
            by='output_variable', 
            inplace=True, 
            ignore_index=True
        )
        percentile_df['output_variable'] = percentile_df['output_variable'].apply(
            lambda x: format_number_suffix(x) + " Percentile"
        )

        sorted_df = pd.concat(
            [non_percentile_df, percentile_df], 
            ignore_index=True
        )

        sorted_df['percentage_difference'] = (
            (sorted_df['test_value'] - sorted_df['train_value']) / 
            sorted_df['train_value']
        ) * 100
        sorted_df['percentage_difference'] = sorted_df['percentage_difference'].apply(
            lambda x: int(x) if not np.isnan(x) and not np.isinf(x) else np.nan
        )

        sorted_df.rename(
            columns={
                'output_variable': 'Distribution Statistic', 
                'train_value': 'Train Value', 
                'test_value': 'Test Value',
                'percentage_difference': '% Difference'
            }, 
            inplace=True
        )

        sorted_df['Train Value'] = sorted_df['Train Value'].map(format_with_commas)
        sorted_df['Test Value'] = sorted_df['Test Value'].map(format_with_commas)

        to_append_df = sorted_df.copy()
        to_append_df['Model Type'] = model_type
        to_append_df['Int/Ext'] = int_or_ext
        to_append_df['DDS Metric'] = dds_metric
        to_append_df['Filename'] = new_filename
        to_append_df['Error Metric'] = error_metric

        new_df = pd.concat([new_df, to_append_df], ignore_index=True)

        tex_table = add_borders_to_tex_table(
            sorted_df.to_latex(
                index=False, 
                caption=title.replace("_", "\\_"), 
                label=f"tab:{title.replace(' ', '_').lower()}",
                column_format='crr',
                position='h',
            ),
            error_metric
        )

        latex_file_output += tex_table + "\n\n"
        

KeyError: 'dds_metric'